In [2]:
import pandas as pd
import os
import numpy as np
import sys
sys.path.append('..')

In [4]:
data = pd.read_csv('../data/interim/all_train_data.csv')

In [8]:
import gc

In [12]:
#data.height_percentage.value_counts()
gc.collect()

0

In [8]:
data.columns

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
     

In [381]:
pct = np.percentile(data.loc[:, 'area_percentage'].fillna(np.mean(data.loc[:, 'area_percentage'])), 97)
print(pct)
print(data.shape)
data = data.loc[data.loc[:, 'area_percentage'] < pct]
print(data.shape)

18.0
(260601, 39)
(252139, 39)


In [382]:
pct = np.percentile(data.loc[:, 'height_percentage'].fillna(np.mean(data.loc[:, 'height_percentage'])), 97)
print(pct)
print(data.shape)
data = data.loc[data.loc[:, 'height_percentage'] < pct]
print(data.shape)

9.0
(252139, 39)
(240799, 39)


In [384]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class GeoInteractionTransformer(BaseEstimator, TransformerMixin):
    """
    Custom transformer to create geo interaction terms by concatenating the geo-level IDs.
    """
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_new = X.copy()
        # Concatenate geo_level_1_id, geo_level_2_id, and geo_level_3_id
        X_new['geo1_geo2'] = X_new['geo_level_1_id'].astype(str) + '_' + X_new['geo_level_2_id'].astype(str)
        X_new['geo1_geo3'] = X_new['geo_level_1_id'].astype(str) + '_' + X_new['geo_level_3_id'].astype(str)
        X_new['geo2_geo3'] = X_new['geo_level_2_id'].astype(str) + '_' + X_new['geo_level_3_id'].astype(str)
        X_new['geo_all'] = (
            X_new['geo_level_1_id'].astype(str) + '_' + 
            X_new['geo_level_2_id'].astype(str) + '_' +
            X_new['geo_level_3_id'].astype(str)
        )
        # Return the entire dataframe including original and new columns
        return X_new

In [385]:
numerical_df = data.select_dtypes(exclude=['object'])
numerical_df.describe()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
count,240799.000000,240799.000000,240799.000000,240799.000000,240799.000000,240799.000000,240799.000000,240799.000000,240799.000000,240799.000000,...,240799.000000,240799.000000,240799.000000,240799.000000,240799.000000,240799.000000,240799.000000,240799.000000,240799.000000,240799.000000
mean,13.853758,702.139207,6276.924360,2.053489,25.865037,7.418868,5.160956,0.081072,0.797292,0.035121,...,0.024477,0.005075,0.000544,0.000174,0.000939,0.000133,0.000087,0.000083,0.004842,2.254665
std,7.938389,409.412468,3646.519437,0.624599,72.641948,2.973722,1.460206,0.272946,0.402018,0.184085,...,0.154525,0.071057,0.023318,0.013206,0.030621,0.011527,0.009338,0.009113,0.069417,0.603874
min,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,7.000000,355.000000,3098.500000,2.000000,10.000000,5.000000,4.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,12.000000,706.000000,6289.000000,2.000000,15.000000,7.000000,5.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,21.000000,1050.000000,9440.000000,2.000000,30.000000,9.000000,6.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,30.000000,1427.000000,12567.000000,9.000000,995.000000,17.000000,8.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000


In [386]:
from scipy.stats import skew

def get_right_skewed_columns(df, skew_threshold=0.5):
    """
    Returns the names of columns that are right-skewed based on the skewness value, excluding binary columns.
    
    Parameters:
    - df: The input DataFrame (numerical columns only).
    - skew_threshold: The skewness threshold above which a column is considered right-skewed (default is 0.5).
    
    Returns:
    - List of column names that are right-skewed.
    """
    right_skewed_columns = []
    
    # Iterate through each column in the dataframe
    for col in df.columns:
        # Check if the column has more than 2 unique values (to avoid binary columns)
        if df[col].nunique() > 2:
            # Calculate skewness for each column
            col_skewness = skew(df[col].dropna())  # Drop NaN values to avoid issues
            
            # Check if the skewness is above the specified threshold (indicating right-skewness)
            if col_skewness > skew_threshold:
                right_skewed_columns.append(col)
    
    return right_skewed_columns


In [390]:
# # Select numerical columns
# numerical_df = data.select_dtypes(exclude=['object'])

# # Get the right-skewed columns
right_skewed_cols = get_right_skewed_columns(numerical_df)

print("Right-skewed columns:", right_skewed_cols)


Right-skewed columns: ['age', 'area_percentage', 'count_families']


In [391]:
from sklearn.preprocessing import FunctionTransformer

def log_transform(X):
    # Apply log1p transformation (log(1 + x)) to avoid issues with zero values
    return np.log1p(X)

# Create a FunctionTransformer for log transformation
log_transformer = FunctionTransformer(log_transform)


In [392]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# Custom transformer for the age-based transformation
class AgeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, age_column='age'):
        self.age_column = age_column
        self.percentile_ = None

    def fit(self, X, y=None):
        # Calculate the 99th percentile of the 'age' column and store it
        self.percentile_ = np.percentile(X[self.age_column].fillna(np.mean(X[self.age_column])), 99)
        return self

    def transform(self, X):
        X_copy = X.copy()
        
        # Add a new 'old' column to indicate if the age exceeds the 99th percentile
        X_copy['old'] = np.where(X_copy[self.age_column] >= self.percentile_, 1, 0)
        
        # Cap the age to 100 where the 'old' column is 1
        X_copy.loc[X_copy['old'] == 1, self.age_column] = 100
        
        return X_copy


In [393]:
x = data.drop(columns=['damage_grade'])
y = data.damage_grade

In [395]:
y = y.replace({1: 0, 2: 1, 3: 2})

In [396]:
y

0         2
1         1
2         2
3         1
5         1
         ..
260596    1
260597    2
260598    2
260599    1
260600    2
Name: damage_grade, Length: 240799, dtype: int64

In [397]:
y.value_counts()

damage_grade
1    137372
2     82375
0     21052
Name: count, dtype: int64

In [400]:
#x.old.value_counts()

In [403]:
#numerical_df = data.select_dtypes(exclude=['object'])
#categorical_df = data.select_dtypes(include=['object'])

In [404]:
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in x.columns if  x[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in x.columns if x[cname].dtype in ['int32', 'int64', 'float64']]

In [405]:
categorical_cols

['land_surface_condition',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'other_floor_type',
 'position',
 'plan_configuration',
 'legal_ownership_status']

In [406]:
#plt.hist(numerical_df.age)

In [407]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler, MinMaxScaler

#train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, shuffle=True)

In [411]:
y_train.value_counts()

damage_grade
1    96394
2    57400
0    14765
Name: count, dtype: int64

In [415]:
y_test.value_counts()

damage_grade
1    40978
2    24975
0     6287
Name: count, dtype: int64

In [416]:
from category_encoders import BaseNEncoder

In [418]:
from sklearn.base import BaseEstimator, TransformerMixin

class SuperstructureFeatureTransformer(BaseEstimator, TransformerMixin):
    """
    Custom transformer to split superstructure-related columns into new features 
    like 'has_stone', 'has_brick', 'has_mortar', 'has_cement', and 'has_mud'.
    """
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        
        # Create the new "has_stone" feature
        X_copy['has_stone'] = (
            X_copy['has_superstructure_mud_mortar_stone'] | 
            X_copy['has_superstructure_stone_flag'] | 
            X_copy['has_superstructure_cement_mortar_stone']
        )
        
        # Create the new "has_brick" feature
        X_copy['has_brick'] = (
            X_copy['has_superstructure_mud_mortar_brick'] | 
            X_copy['has_superstructure_cement_mortar_brick']
        )
        
        # Create the new "has_adobe_mud" feature
        #X_copy['has_adobe_mud'] = X_copy['has_superstructure_adobe_mud']
        
        # Create the new "has_mortar" feature
        X_copy['has_mortar'] = (
            X_copy['has_superstructure_mud_mortar_stone'] | 
            X_copy['has_superstructure_mud_mortar_brick']
        )
        
        # Create the new "has_cement" feature
        X_copy['has_cement'] = (
            X_copy['has_superstructure_cement_mortar_stone'] | 
            X_copy['has_superstructure_cement_mortar_brick']
        )
        
        # Create the new "has_mud" feature
        X_copy['has_mud'] = (
            X_copy['has_superstructure_mud_mortar_stone'] | 
            X_copy['has_superstructure_mud_mortar_brick'] | 
            X_copy['has_superstructure_adobe_mud']
        )
        
        return X_copy


In [419]:
# #create numerical transformer
geo_interaction_transformer = GeoInteractionTransformer()

geo_cols = ['geo1_geo2', 'geo1_geo3', 'geo2_geo3', 'geo_all']

# # # BaseNEncoder for the high cardinality of interaction features
base_encoder_geo = BaseNEncoder(cols=geo_cols, base=5)

# # Geo interaction pipeline: First, create interaction terms, then encode them
geo_interaction_pipeline = Pipeline(steps=[
     ('geo_interaction', geo_interaction_transformer),  # Create interaction terms
     ('base_encoder', base_encoder_geo)  # Encode interaction terms
])


# numerical_transformer = Pipeline([('imputer', SimpleImputer(strategy='mean'))])

# #create categorical transformer
# #categorical_transformer = Pipeline(steps=[ ('imputer', SimpleImputer(strategy='most_frequent')),
# #                                            ('onehot', OneHotEncoder(handle_unknown='ignore'))
# #                                            ])

#reduction_columns = ['geo_level_1_id', 'geo_level_2_id','geo_level_3_id']


# base_encoder = Pipeline(steps=[
#     ('base_encoder', BaseNEncoder(cols=reduction_columns, base=3))
# ])

# age_transformer = Pipeline(steps=[
#     ('age_transform', AgeTransformer(age_column='age'))  # Apply age transformation
# ])

In [319]:
data.columns

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
     

In [440]:
import torch
import torch.nn as nn
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import pandas as pd

# Define the autoencoder class
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim)
        )
        # Decoder (optional, only necessary if you want reconstruction)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Custom transformer to handle autoencoder training and dimensionality reduction
class AutoencoderTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, latent_dim=10, epochs=100, batch_size=32, learning_rate=1e-3):
        self.latent_dim = latent_dim
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.autoencoder = None  # Autoencoder model will be initialized in `fit`

    def fit(self, X, y=None):
        # Automatically determine input_dim based on X's shape after BaseN encoding
        input_dim = X.shape[1]  # Number of features in X after BaseN encoding
        self.autoencoder = Autoencoder(input_dim=input_dim, latent_dim=self.latent_dim)

        # Convert X to NumPy array if it's a DataFrame
        if isinstance(X, pd.DataFrame):
            X = X.values

        X_tensor = torch.tensor(X, dtype=torch.float32)  # Convert NumPy array to tensor
        optimizer = torch.optim.Adam(self.autoencoder.parameters(), lr=self.learning_rate)
        criterion = nn.MSELoss()

        # Training loop
        for epoch in range(self.epochs):
            optimizer.zero_grad()
            encoded, decoded = self.autoencoder(X_tensor)
            loss = criterion(decoded, X_tensor)
            loss.backward()
            optimizer.step()

            if epoch % 10 == 0:  # Print loss every 10 epochs for monitoring
                print(f'Epoch {epoch}, Loss: {loss.item()}')
                
        return self

    def transform(self, X):
        # Convert X to NumPy array if it's a DataFrame
        if isinstance(X, pd.DataFrame):
            X = X.values

        self.autoencoder.eval()  # Set to evaluation mode
        X_tensor = torch.tensor(X, dtype=torch.float32)  # Convert NumPy array to tensor
        with torch.no_grad():
            encoded, _ = self.autoencoder(X_tensor)
        return encoded.numpy()  # Return the reduced-dimensional representation


reduction_columns = ['geo_level_1_id', 'geo_level_2_id','geo_level_3_id']

base_encoder_columns = ['land_surface_condition',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'other_floor_type',
 'position',
 'plan_configuration',
 'legal_ownership_status']

base_encoder = Pipeline(steps=[
    ('base_encoder', BaseNEncoder(cols=base_encoder_columns, base=3))
])

base_encoder_geo = Pipeline(steps=[
    ('base_encoder', BaseNEncoder(cols=reduction_columns, base=3))
])

# Define the pipeline with an Autoencoder for dimensionality reduction
preprocessor = ColumnTransformer(transformers=[

    ('base_name_geo', base_encoder_geo, reduction_columns),  # BaseN encoding on categorical columns

    # Autoencoder dimensionality reduction for BaseN encoded features
    ('autoencoder', Pipeline(steps=[
        ('extractor', FunctionTransformer(lambda x: x, feature_names_out='one-to-one')),  # Pass through BaseN encoded features
        ('autoencoder', AutoencoderTransformer(latent_dim=5))  # Autoencoder with latent dimension 10
    ]), reduction_columns),
    # ('autoencoder', Pipeline(steps=[
    #     ('extractor', FunctionTransformer(lambda x: x, feature_names_out='one-to-one')),  # Extract original geo columns
    #     ('autoencoder', AutoencoderTransformer(latent_dim=5))  # Autoencoder for geo features with latent dimension 5
    # ]), reduction_columns),

    ('base_name', base_encoder, base_encoder_columns),  # BaseN encoding on categorical columns
    ('age_transform', age_transformer, ['age']),  # Custom transformer for 'age'
    ('num', 'passthrough', numerical_cols),  # Pass through numerical columns without transformation
    ('log_transform', log_transformer, right_skewed_cols)  # Log transform for right-skewed columns
    # Add other transformers or steps as needed
])




In [442]:
# # Define the pipeline with an Autoencoder replacing PCA
# preprocessor = ColumnTransformer(transformers=[
#     ('base_name', base_encoder, base_encoder_columns),  # BaseN encoding on categorical columns

#     # Autoencoder dimensionality reduction for BaseN encoded features
#     ('autoencoder', Pipeline(steps=[
#         ('extractor', FunctionTransformer(lambda x: x, feature_names_out='one-to-one')),  # Extract BaseN encoded features
#         ('autoencoder', AutoencoderTransformer(input_dim=15, latent_dim=10))  # Adjust input_dim to match BaseN output
#     ]), base_encoder_columns),

#     ('age_transform', age_transformer, ['age']),  # Custom transformer for 'age'
#     ('num', 'passthrough', numerical_cols),  # Pass through numerical columns without transformation
#     ('log_transform', log_transformer, right_skewed_cols)  # Log transform for right-skewed columns
# ])


In [444]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

# LightGBM for Multiclass Classification
lgbm = LGBMClassifier(
    n_estimators=843,
    learning_rate=0.16934236113095621,
    max_depth=7,
    random_state=42,
    lambda_l1=7.455101924325866,  # L2 regularization
    lambda_l2=0.007915220427757011,
    subsample=0.7354468519750788,    # Subsample ratio of the training instances
    colsample_bytree=0.8648222301238216,           # Subsample ratio of columns when constructing each tree
    min_child_weight=2,             # Equivalent of min_data_in_leaf in LightGBM
    subsample_for_bin=200000,
    num_leaves=31,
    #reg_alpha=0.14170716330946964,  # L1 regularization term
    objective='multiclass',         # Objective for multiclass classification
    metric='multi_logloss',         # Metric used for multiclass classification
    num_class=3                     # Specify the number of classes in the target
)

rf_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Step 1: Preprocessing
    ('xgboost', lgbm)  # Step 3: Model training
])

# Preprocessing of training data, fit model 
#rf_pipe.fit(X_train, y_train)

# Preprocessing of training data, fit model after upsampling!
rf_pipe.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
rf_preds = rf_pipe.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, rf_preds)
print('Accuracy for XGBoost:', accuracy)

# Detailed classification report
print('Classification Report:\n', classification_report(y_test, rf_preds))

C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfi

Epoch 0, Loss: 19167194.0
Epoch 10, Loss: 15490962.0
Epoch 20, Loss: 9518197.0
Epoch 30, Loss: 2524483.75
Epoch 40, Loss: 303220.125
Epoch 50, Loss: 407640.03125
Epoch 60, Loss: 122995.921875
Epoch 70, Loss: 114088.3046875
Epoch 80, Loss: 57516.64453125
Epoch 90, Loss: 31230.220703125
[LightGBM] [Warning] lambda_l1 is set=7.455101924325866, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.455101924325866
[LightGBM] [Warning] lambda_l2 is set=0.007915220427757011, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.007915220427757011
[LightGBM] [Warning] lambda_l1 is set=7.455101924325866, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.455101924325866
[LightGBM] [Warning] lambda_l2 is set=0.007915220427757011, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.007915220427757011
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030897 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory 

C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfi

[LightGBM] [Warning] lambda_l1 is set=7.455101924325866, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.455101924325866
[LightGBM] [Warning] lambda_l2 is set=0.007915220427757011, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.007915220427757011
Accuracy for XGBoost: 0.7463178294573644
Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.50      0.57      6287
           1       0.75      0.85      0.79     40978
           2       0.76      0.64      0.70     24975

    accuracy                           0.75     72240
   macro avg       0.73      0.66      0.69     72240
weighted avg       0.75      0.75      0.74     72240



In [345]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'preprocessor__autoencoder__autoencoder__latent_dim': [5, 10, 15, 20]  # Testing different latent_dim values
# }

# # Create a grid search
# grid_search = GridSearchCV(rf_pipe, param_grid, cv=3)
# grid_search.fit(X_train, y_train)

# # Best latent_dim
# best_latent_dim = grid_search.best_params_['preprocessor__autoencoder__autoencoder__latent_dim']
# print(f'Best latent_dim: {best_latent_dim}')

In [347]:
# from sklearn.metrics import accuracy_score, classification_report
# from xgboost import XGBClassifier
# # XGBoost
# xgb = XGBClassifier(
#     n_estimators=600,
#     learning_rate=0.2669112505018992,
#     max_depth=5,
#     random_state=42,
#     reg_lambda=1.2259716591605452,
#     subsample=0.704976942819638,
#     colsample_bytree=0.9,
#     min_child_weight=4,
#     alpha= 0.14170716330946964,    # Added L1 regularization
#     eval_metric='mlogloss',  # Consider custom loss for ordinal
#     objective='multi:softmax',  # Using softmax but can tweak for ordinal
#     num_class=3  # Assuming 3 ordinal classes
# )

# rf_pipe = Pipeline(steps=[
#     ('preprocessor', preprocessor),  # Step 1: Preprocessing
#     ('xgboost', xgb)  # Step 3: Model training
# ])

# # Preprocessing of training data, fit model 
# #rf_pipe.fit(X_train, y_train)

# # Preprocessing of training data, fit model after upsampling!
# rf_pipe.fit(X_train, y_train)

# # Preprocessing of validation data, get predictions
# rf_preds = rf_pipe.predict(X_test)

# # Evaluate the model
# accuracy = accuracy_score(y_test, rf_preds)
# print('Accuracy for XGBoost:', accuracy)

# # Detailed classification report
# print('Classification Report:\n', classification_report(y_test, rf_preds))

C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfi

Epoch 0, Loss: 17895082.0
Epoch 10, Loss: 10108766.0
Epoch 20, Loss: 2132321.25
Epoch 30, Loss: 528997.9375
Epoch 40, Loss: 304072.46875
Epoch 50, Loss: 179036.859375
Epoch 60, Loss: 94923.5546875
Epoch 70, Loss: 79697.3828125
Epoch 80, Loss: 52537.9140625
Epoch 90, Loss: 42737.546875


C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfi

Accuracy for XGBoost: 0.7441860465116279
Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.50      0.57      6287
           1       0.74      0.85      0.79     40978
           2       0.76      0.64      0.69     24975

    accuracy                           0.74     72240
   macro avg       0.73      0.66      0.69     72240
weighted avg       0.74      0.74      0.74     72240



In [353]:
# Preprocessing of validation data, get predictions
rf_preds = rf_pipe.predict(X_train)

# Evaluate the model
accuracy = accuracy_score(y_train, rf_preds)
print('Accuracy for XGBoost:', accuracy)

# Detailed classification report
print('Classification Report:\n', classification_report(y_train, rf_preds))

C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfi

[LightGBM] [Warning] Unknown parameter: lamda_l2
[LightGBM] [Warning] lambda_l1 is set=7.455101924325866, reg_alpha=0.14170716330946964 will be ignored. Current value: lambda_l1=7.455101924325866
Accuracy for XGBoost: 0.7974833737741681
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.60      0.69     14765
           1       0.79      0.89      0.84     96394
           2       0.82      0.69      0.75     57400

    accuracy                           0.80    168559
   macro avg       0.80      0.73      0.76    168559
weighted avg       0.80      0.80      0.79    168559



In [359]:
test_data = pd.read_csv('../data/raw/test_values.csv')

In [361]:
X_test_final = test_data.building_id

In [363]:
X_test_final

0         300051
1          99355
2         890251
3         745817
4         421793
          ...   
86863     310028
86864     663567
86865    1049160
86866     442785
86867     501372
Name: building_id, Length: 86868, dtype: int64

In [365]:
rf_preds = rf_pipe.predict(test_data)
rf_preds = pd.Series(rf_preds)
rf_preds 

C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\Administrator\anaconda3\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfi

[LightGBM] [Warning] lambda_l1 is set=7.455101924325866, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.455101924325866
[LightGBM] [Warning] lambda_l2 is set=0.007915220427757011, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.007915220427757011


0        2
1        1
2        1
3        0
4        2
        ..
86863    1
86864    2
86865    1
86866    1
86867    0
Length: 86868, dtype: int64

In [367]:
df_concatenated = pd.concat([X_test_final, rf_preds], axis=1)
df_concatenated
df_concatenated = df_concatenated.rename(columns={0: 'damage_grade'})
df_concatenated
df_concatenated['damage_grade'] = df_concatenated['damage_grade'].replace({0: 1, 1: 2, 2: 3})
df_concatenated.to_csv('../data/processed/RLsub1.csv', index=False)